In [158]:
#import pacakges 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import math
import numpy
import cv2
from pylab import *
from numpy import *
from numpy import array
import random;
from scipy.interpolate import RectBivariateSpline

# 1) Use ginput to find correspondances

In [316]:
def getPoints(image):
    img1 = Image.open(image) 
    plt.figure(1)
    plt.imshow(img1)
    print("Please click") 
    x = plt.ginput(4)
    return x

In [317]:
pts_src = getPoints('./Images/image1.jpg')
print(pts_src)
pts_dst=getPoints('./Images/image2.jpg')
print(pts_dst)
plt.close()

Please click


/usr/lib/python3/dist-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


[(780.7925483121329, 539.818493150685), (886.0493976272015, 325.12793542074377), (1018.0381451810175, 331.8109099804306), (907.7690649461839, 540.6538649706458)]
Please click
[(324.67953461350294, 505.5682485322897), (442.46696122798437, 301.73752446183966), (559.4190160225048, 317.609589041096), (446.6438203277886, 509.745107632094)]


In [318]:
#check sizes of image one and image 2
img2 = array(Image.open('./Images/image2.jpg'))
print(img2.shape)
img1 = array(Image.open('./Images/image1.jpg'))
print (img1.shape)

(683, 1024, 3)
(683, 1024, 3)


#

In [351]:
#get x,y,x',y' arrays from cores
def getArrays(src,dest):
    x=[]
    y=[]
    xdash=[]
    ydash=[]

    for i in range(4):
        y.append(src[i][0])
        x.append(src[i][1])
        ydash.append(dest[i][0])
        xdash.append(dest[i][1])

    return x,y,xdash,ydash

In [352]:
#homography parameters implemntation
def findHomography (src, dest):
    b = dest.flatten().reshape(-1,1)

    x,y,xdash,ydash = getArrays(src,dest)

    M = np.array([
          [x[0],y[0],1,0,0,0,-x[0]*xdash[0], -y[0]*xdash[0]],
          [0,0,0,x[0],y[0],1,-x[0]*ydash[0], -y[0]*ydash[0]],
          [x[1],y[1],1,0,0,0,-x[1]*xdash[1],-y[1]*xdash[1]],
          [0,0,0,x[1],y[1],1,-x[1]*ydash[1],-y[1]*ydash[1]],
          [x[2],y[2],1,0,0,0,-x[2]*xdash[2],-y[2]*xdash[2]],
          [0,0,0,x[2],y[2],1,-x[2]*ydash[2],-y[2]*ydash[2]],
          [x[3],y[3],1,0,0,0,-x[3]*xdash[3],-y[3]*xdash[3]],
          [0,0,0,x[3],y[3],1,-x[3]*ydash[3],-y[3]*ydash[3]]
      ])
    a = np.dot(np.linalg.inv(M),b)
    a = np.append(a,1)
    return a.reshape(3,3)

In [353]:
#test to check homography function
pts_src_test = np.array([[141, 131], [480, 159], [493, 630],[64, 601]])
pts_dst_test = np.array([[318, 256],[534, 372],[316, 670],[73, 473]])
h_test=findHomography(pts_src_test,pts_dst_test)
print("calculated homography", h_test)
h_test, status_tst = cv2.findHomography(pts_src_test, pts_dst_test)
print("actual homogrpahy", h_test)


calculated homography [[-4.46961746e-01  8.21391529e-01  2.70565473e+02]
 [ 5.14312555e-01  5.16763310e-01  1.27971747e+02]
 [-1.04366138e-04  3.69285047e-04  1.00000000e+00]]
actual homogrpahy [[ 4.34043935e-01 -4.19622184e-01  2.91709494e+02]
 [ 1.46491654e-01  4.41418278e-01  1.61369294e+02]
 [-3.62463336e-04 -9.14274844e-05  1.00000000e+00]]


In [354]:
h=findHomography(np.array(pts_dst),np.array(pts_src))
print("calculated homography", h)
h, status = cv2.findHomography(np.array(pts_dst),np.array(pts_src))
print("actual homogrpahy", h)

calculated homography [[ 3.61064622e-01  1.19956996e+00  3.06319342e+02]
 [ 1.19697934e+00  4.38747953e-01 -6.66992129e+01]
 [ 1.59469058e-04  3.08235957e-04  1.00000000e+00]]
actual homogrpahy [[ 7.63504839e-01  1.38385778e-01  4.27004668e+02]
 [-1.61048095e-01  9.81394155e-01  7.11047588e+01]
 [-2.50182943e-04  6.96481766e-05  1.00000000e+00]]


# Warping the source image to the destination image

## Forward warping 

In [355]:
def correct(A):
    if A[2]!=1:
        A[0] = (A[0]/A[2])
        A[1] = (A[1]/A[2])
        A[2] = (A[2]/A[2])
    return A
def round_arr(A):
    A[0]= round(A[0])
    A[1]= round(A[1])
    A[2]= round(A[2])
    return A  

In [356]:
def Get_max_min_Axis (A,B,C,D):
    
    min_x = min(A[0],B[0],C[0],D[0])
    max_x = max(A[0],B[0],C[0],D[0])
    min_y = min(A[1],B[1],C[1],D[1])
    max_y = max(A[1],B[1],C[1],D[1])
   
    return min_x , max_x , min_y , max_y

In [357]:
def Get_new_boundaries (h,N,M):
    A= [0,0,1]
    A_new  = np.dot(h,A)
    A_new = correct(A_new)
    A_new = round_arr(A_new)
    print("old value of A = ",A,"new value = ",A_new)
    B=[682,0,1]
    B_new = np.matmul(h,B )
    B_new = correct(B_new)
    B_new = round_arr(B_new)
    C=[0,1023,1]
    print("old value of B = ",B,"new value = ",B_new)
    C_new = np.matmul(h,C )
    C_new = correct(C_new)
    C_new = round_arr(C_new)
    D=[682,1023,1]
    print("old value of C= ",C,"new value = ",C_new)
    D_new = np.matmul(h,D )
    D_new = correct(D_new)
    D_new = round_arr(D_new)
    print("old value of D = ",D,"new value = ",D_new)
    #print(A_new,B_new,C_new,D_new)
    return A_new,B_new,C_new,D_new

In [358]:
A,B,C,D = Get_new_boundaries(h,img2.shape[0],img2.shape[1])

old value of A =  [0, 0, 1] new value =  [427.  71.   1.]
old value of B =  [682, 0, 1] new value =  [ 1.143e+03 -4.700e+01  1.000e+00]
old value of C=  [0, 1023, 1] new value =  [5.310e+02 1.004e+03 1.000e+00]
old value of D =  [682, 1023, 1] new value =  [1.209e+03 1.072e+03 1.000e+00]


In [359]:
min_x , max_x , min_y, max_y = Get_max_min_Axis (A,B,C,D)
print(min_x , max_x , min_y, max_y)

427.0 1209.0 -47.0 1072.0


In [360]:
new_h = int(round(abs(max_x - min_x)))
new_w = int(round(abs(max_y - min_y)))
print("the new width is ",new_w,"the new height is",new_h)

the new width is  1119 the new height is 782


## Inverse warping

In [375]:
#compute inverse homography
h_inv = np.linalg.inv(h)

In [376]:
#convert the image to greyfor trial
gray= cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

In [377]:
#prepare x and y arrays for interpolation
x = np.arange(0, img2.shape[0], 1)
y = np.arange(0, img2.shape[1], 1)
interp_spline = RectBivariateSpline(x.reshape(-1,1), y.reshape(-1,1), gray)

In [378]:
new_img = np.zeros((new_h,new_w))
l=[]
for i in range (int(min_x),int(min_x+new_h),1):
    for j in range (int(min_y),int((min_y+new_w)),1):
        point = [i,j,1]
        co_ord = np.matmul(h_inv,point)
        co_ord = correct(co_ord)
        xi=co_ord[0]
        yi=co_ord[1]
        if (xi>=0 and yi>=0):
            intensity=int(interp_spline.ev(xi, yi))
            l.append(intensity)
        else:
            l.append(0)

In [389]:
#show the warped image in grey scale
warped_image = np.array(l).reshape(new_h,new_w)
print("width",warped_image.shape[0],"height",warped_image.shape[1])
plt.imshow(warped_image,cmap='gray')
plt.show()

width 782 height 1119


# Stitch the two images together

In [398]:
result = np.zeros((img1.shape[0]+new_h,warped_image.shape[1]+new_w))
print(result.shape)
a = int (A[0])
b = int (A[1])
result[b:(b+782),a:(a+1119)]=warped_image
print(r.shape)
plt.imshow(result,cmap='gray')
plt.show()
plt.close()


(1465, 2238)
(777, 1110)


In [399]:
gray_2= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
result[0:img1.shape[0],0:img1.shape[1]]=gray_2
plt.imshow(result,cmap='gray')
plt.show()
plt.close()

In [ ]:
dst = cv2.warpPerspective(img2,h,(img1.shape[1] + img2.shape[1], img1.shape[0]))
print (h)
print(dst.shape)
figure(2)
plt.imshow(dst,cmap='gray')
plt.show()
plt.close()
dst[0:img1.shape[0], 0:img1.shape[1]] = img1
plt.imshow(dst)
plt.show()

[[ 7.63504839e-01  1.38385778e-01  4.27004668e+02]
 [-1.61048095e-01  9.81394155e-01  7.11047588e+01]
 [-2.50182943e-04  6.96481766e-05  1.00000000e+00]]
(683, 2048, 3)
